In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import requests
from openpyxl import *
import pandas as pd
import subprocess
import re
from urllib.request import urlopen

In [3]:
options = Options()
##
# 크롬 브라우저 동작방식 설정
options.add_argument("--start-maximized") # 브라우저 전체화면
options.add_experimental_option("detach", True) # 셀레니움이 종료되어도 창이 자동으로 닫히지 않게 함.
options.add_argument("--disable-blink-features=AutomationControlled")
##
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

url = 'http://data.prosports.or.kr/page/spectator/date'

driver.get(url)
time.sleep(2)

In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

# html 확인/ 필요한 정보가 들어있는 태그 추출
temperature1 = soup.find_all('div','filt_th') # 내장함수 find가 아님
table = soup.find('div', 'scrollBody') # 테이블들의 데이터 가져오기기


# 테이블 데이터 행 찾기 (모든 <tr> 태그)
rows = table.find_all('tr')

# 데이터를 담을 리스트
data = []

# 각 행에서 데이터 추출
for row in rows:
    # 헤더 행(<th>)은 건너뛰기
    if row.find('th'):
        continue
    
    # 데이터 행의 <td> 태그 찾기
    cols = row.find_all('td')
    # <td> 안의 <p> 태그의 텍스트를 가져오기
    cols = [col.find('p').text.strip() if col.find('p') else '' for col in cols]
    if any(cols):  # 값이 하나라도 존재하는 경우에만 추가
        data.append(cols)

# 열이름 가져오기기
columns = [col_1.p.text.strip() for col_1 in temperature1]

# 데이터 값들 가져오기기
data_dict = [dict(zip(columns, row)) for row in data]

# 데이터 프레임으로 변환환
df = pd.DataFrame(data_dict)

# 데이터 형태 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1742 entries, 0 to 1741
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      1742 non-null   object
 1   시간      1742 non-null   object
 2   단체/구단   1742 non-null   object
 3   경기      1742 non-null   object
 4   장소      1742 non-null   object
 5   결과      1742 non-null   object
 6   날씨      1742 non-null   object
 7   관중수     1742 non-null   object
dtypes: object(8)
memory usage: 109.0+ KB


In [10]:
# CSV 파일로 저장 (utf-8-sig 인코딩 적용)
df.to_csv('종목별_장소_관중수.csv', index=False, encoding='utf-8-sig')